In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp
import yaml

import random
from itertools import chain, combinations, tee
import time


# Functions

In [20]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]


# Groups, Routes to Edges:

In [2]:
directory_path = '../data/data_income_percentage_VoT/'
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')

dict_data = {}

# df_od_flow_data
# df_data

In [3]:
for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])


In [4]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 617.3087999,
  'Demand (eligible group, 1)': 48.15008639220001,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 27.7788959955,
  'VoT (eligible group, 2)': 0.10238603988603988,
  'Demand (ineligible group, 1)': 116.05405438120002,
  'VoT (ineligible group, 1)': 0.2754407051282051,
  'Demand (ineligible group, 2)': 135.80793597800002,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 289.51782715310003,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 92.22995123,
  'Demand (eligible group, 1)': 7.19393619594,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 4.15034780535,
  'VoT (eligible group, 2)': 0.1023860398

In [9]:
cities_dict = {}
for od in list(dict_data.values()):
    if od["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od["Start City Index"]] = od["Start City"]
    if od["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od["End City Index"]] = od["End City"]

cities_list = list(cities_dict.values())

In [13]:
# print(list(cities_dict.keys()))
# cities_list
# list(dict_data.keys())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [30]:
demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array[od_index, group_index] = od_value[VoT_name]

# demand_array
# VoT_array

# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [ ]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


In [32]:
def welfare_obj(T, num_edges, lambda_E, lambda_R, lambda_I, tau, \
                demand_array, VoT_array, num_el, y, od_edges_array, \
                a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    assert len(od_edges_array.shape) == 2, "od_edges should be 2-dimensional."
    assert od_edges_array.shape[1] == 2, "od_edges' second dimension should be for start and end edges."
    
    edges_to_od_dict = {}
    for edge in range(num_edges):
        edges_to_od_dict[e] = [k for k in list(range(od_edges_array.shape[0])) \
                               if od_edges_array[k, 0] <= e <= od_edges_array[k, 1]]
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, len(E)))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, len(E)))
        
    a = np.zeros((a_ex.shape[0], a_ex.shape[1], 2))
    a[:, :, 0] = a_ex
    a[:, :, 1] = a_gp
    
    # In full:
    # y indices: (od, group, edge, "lane", time)
    # ("group" includes info on whether said group is eligible.)
    
    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edges_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edges_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edges_to_od_dict[e] \
                              for g in in_indices)
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            for k in range(2):
                ell[e, k, t] = sum(a[p, e, k] * (x[e, k, t] ** p) for p in range(5))

    obj_E = sum( y[(od, g, e, 0, t)] * VoT_array[od, g] * ell[e, 0, t] \
                for e in range(num_edges) for od in edges_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * (VoT_array[od, g] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edges_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 2, t)] * VoT_array[od, g] * ell[e, 1, t] \
                  for e in range(num_edges) for od in edges_to_od_dict[e] \
                  for g in el_indices for t in range(T) )
    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edges_to_od_dict[e] \
                for g in in_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g] * ell[e, 1, t] \
                for e in range(num_edges) for od in edges_to_od_dict[e] \
                  for g in in_indices for t in range(T) )
    obj_R = sum( y[(od, g, e, 0, t)] * tau[e, t] \
                for e in range(num_edges) for od in edges_to_od_dict[e] \
                for g in in_indices for t in range(T) )

    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare


# Latency, total, throughout the entire time horizon. 

def latency_total(T, num_edges, lambda_E, lambda_R, lambda_I, tau, \
                  demand_array, VoT_array, num_el, y, od_edges_array, \
                  a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                  a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    assert len(od_edges_array.shape) == 2, "od_edges should be 2-dimensional."
    assert od_edges_array.shape[1] == 2, "od_edges' second dimension should be for start and end edges."
    
    edges_to_od_dict = {}
    for edge in range(num_edges):
        edges_to_od_dict[e] = [k for k in list(range(od_edges_array.shape[0])) \
                               if od_edges_array[k, 0] <= e <= od_edges_array[k, 1]]
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, len(E)))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, len(E)))
        
    a = np.zeros((a_ex.shape[0], a_ex.shape[1], 2))
    a[:, :, 0] = a_ex
    a[:, :, 1] = a_gp
    
    # In full:
    # y indices: (od, group, edge, "lane", time)

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edges_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edges_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edges_to_od_dict[e] \
                              for g in in_indices)
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            for k in range(2):
                ell[e, k, t] = sum(a[p, e, k] * (x[e, k, t] ** p) for p in range(5))

    latency = sum( x[e, k, t] * ell[e, k, t] \
                for e in range(num_edges) for k in range(2) for t in range(T) )

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return latency

## <font color='red'>To edit below</font> 

In [ ]:
# def welfare_obj(T, num_edges, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, y, od_edges_array, \
#                 a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
#                 a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):



def proj_tau_B(T, num_edges, tau, B, od_edges_list_full, tau_max = 1.0, B_max = 1.0):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "toll vector length must equal the number of edges."
    
    tau_feas = cp.Variable((num_edges, T))
    B_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau) + (B_feas - B)**2

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
    constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max * np.ones((num_edges, T))]
    constraints += [B_feas <= B_max]
    constraints += [B_feas <= sum(tau_feas[e, t] for e in route) ]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

#     print()
#     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
#     print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas.value, 0.0), decimals=4)


# Chinmay's Algorithm:

## Convex Program for CBCP and DBCP Equilibria:

In [ ]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [ ]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_CBCP_direct(T, network, tau, B, v_I_array, v_E_array, \
                         a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                         a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    assert np.all(a_ex >= 0.0), "All entries of a_ex must be non-negative."
    assert np.all(a_gp >= 0.0), "All entries of a_gp must be non-negative."
    assert a_ex.shape == a_gp.shape, "a_ex and a_gp must have the same shape."
    assert len(a_ex.shape) in [1, 2], "a_ex and a_gp must be either a vector or a matrix."
    
    if len(a_ex.shape) == 1:
        latency_params_length = a_ex.shape[0]
        a_ex = a_ex.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
        a_gp = a_gp.reshape((latency_params_length, 1)) @ np.ones((1, network.num_edges))
    
    # Variable indices:
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, edge, "lane")
    # y_in indices: (group, edge, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]
    
    # Variables:
    y_el = {}
    for g in range(num_el):
        for e in network.edges:
            for k in [0, 1, 2]:
                for t in range(T):
                    y_el[(g, e, k, t)] = cp.Variable(1)
            
    y_in = {}
    for g in range(num_in):
        for e in network.edges:
            for k in [0, 1]:
                for t in range(T):
                    y_in[(g, e, k, t)] = cp.Variable(1)
    
    x_ex = {}
    x_gp = {}
    for e in network.edges:
        for t in range(T):
            x_ex[(e, t)] = cp.Variable(1)
            x_gp[(e, t)] = cp.Variable(1)
    
    # Objective:
    func = 0.0
    for e in network.edges:
        for t in range(T):
            func += 1/5 * a_ex[4, e] * cp.power(x_ex[(e, t)], 5)
            func += 1/4 * a_ex[3, e] * cp.power(x_ex[(e, t)], 4)
            func += 1/3 * a_ex[2, e] * cp.power(x_ex[(e, t)], 3)
            func += 1/2 * a_ex[1, e] * cp.power(x_ex[(e, t)], 2)
            func += a_ex[0, e] * x_ex[(e, t)]
            func += 1/5 * a_gp[4, e] * cp.power(x_gp[(e, t)], 5)
            func += 1/4 * a_gp[3, e] * cp.power(x_gp[(e, t)], 4)
            func += 1/3 * a_gp[2, e] * cp.power(x_gp[(e, ts)], 3)
            func += 1/2 * a_gp[1, e] * cp.power(x_gp[(e, t)], 2)
            func += a_gp[0, e] * x_gp[(e, t)]

            func += sum(tau[e, t] * y_el[(g, e, 1, t)] / v_E_array[g] for g sfin range(num_el))
            func += sum(tau[e, t] * y_in[(g, e, 0, t)] / v_I_array[g] for g in range(num_in))

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y_el[(g, e, k, t)] >= 0.0 for g in range(num_el) for e in network.edges \
                    for k in [0, 1, 2] for t in range(T)]
    constraints += [y_in[(g, e, k, t)] >= 0.0 for g in range(num_in) for e in network.edges \
                    for k in [0, 1] for t in range(T)]
        
    constraints += [sum(y_el[(g, e, k, t)] for e in network.start_edges for k in [0, 1, 2]) == 1.0 \
                    for g in range(num_el) for t in range(T)]
    constraints += [sum(y_in[(g, e, k, t)] for e in network.start_edges for k in [0, 1]) == 1.0 \
                    for g in range(num_in) for t in range(T)]
    for node in network.intermediate_nodes:
        constraints += [sum(y_el[(g, e, k, t)] for e in network.nodes_dict[node]["incoming_edges"] for k in [0, 1, 2]) \
                        == sum(y_el[(g, e, k, t)] for e in network.nodes_dict[node]["outgoing_edges"] for k in [0, 1, 2]) \
                        for g in range(num_el) for t in range(T)]
        constraints += [sum(y_in[(g, e, k, t)] for e in network.nodes_dict[node]["incoming_edges"] for k in [0, 1]) \
                        == sum(y_in[(g, e, k, t)] for e in network.nodes_dict[node]["outgoing_edges"] for k in [0, 1]) \
                        for g in range(num_in) for t in range(T)]
    
    for e in network.edges:
        for t in range(T):
            constraints += [x_ex[(e, t)] == sum(y_el[(g, e, k, t)] for g in range(num_el) for k in [0, 1]) \
                                       + sum(y_in[(g, e, 0, t)] for g in range(num_in)) ]
            constraints += [x_gp[(e, t)] == sum(y_el[(g, e, 2, t)] for g in range(num_el)) \
                                       + sum(y_in[(g, e, 1, t)] for g in range(num_in)) ]
    
    constraints += [sum(y_el[(g, e, 0, t)] * tau[e, t] for e in network.edges for t in range(T)) \
                    <= B for g in range(num_el)]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
    result = prob.solve()
    
    # Extract Values:
    y_el_values = {}
    for g in range(num_el):
        for e in network.edges:
            for k in [0, 1, 2]:
                for t in range(T):
                    y_el_values[(g, e, k, t)] = max(y_el[(g, e, k, t)].value[0], 0.0)

    y_in_values = {}
    for g in range(num_in):
        for e in network.edges:
            for k in [0, 1]:
                for t in range(T):
                    y_in_values[(g, e, k, t)] = max(y_in[(g, e, k, t)].value[0], 0.0)

    return y_el_values, y_in_values


## Implement Zeroth-order Gradient Descent:

In [ ]:
time_1 = time.time()

# TODO: Edit below to incorporate time

T = 5

# edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3]])
edges_nodes_matrix = np.array([[0, 1], [0, 2], [1, 2], [1, 3], [2, 3], [0, 3]])
network_Braess_augmented = network_structure(edges_nodes_matrix)
# network_Braess.print_all()

network = network_Braess_augmented

tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4, 0.5])
B = 0.8
v_I_array = np.array([1.0, 1.2, 1.4])
v_E_array = np.array([0.6, 0.8, 0.9])
flow_per_group = 1.0
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
# y_init = np.array([0.0, 0.05, 0.95, 0.95, 0.05])

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 10.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# tau_max, B_max = 1.0, 1.0
flow_max = flow_per_group * (v_I_array.shape[0] + v_E_array.shape[0])
tau_max = sum(a[k] * (flow_max ** k) for k in range(5))
B_max = tau_max

d = 2
num_iters = 1000
tau = np.zeros((network.num_edges, T, num_iters))
tau_perturbed = np.zeros((network.num_edges, T, num_iters))

B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 3.0
delta_bar = 3.0

welfare_list = []

# tau[:, :, 0] = tau_max * 0.8
# B[0] = B_max * 0.2

tau[:, :, 0] = tau_max * 0.7
B[0] = B_max * 0.3

# tau[:, :, 0] = tau_max * 0.6
# B[0] = B_max * 0.4

# TODO:
# Rename od_edges_array to match above definition.

od_edges_list_full = [list(range(od_edges_array[k, 0], od_edges_array[k, 1])) \
                      for k in range(od_edges_array.shape[0])]

for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(network.num_edges * T + 1)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
    print("w_i:", w_i)
    tau_perturbed[:, :, i] = tau[:, :, i] + delta[i] * w_i[:-1].reshape((network.num_edges, T))
    B_perturbed[i] = B[i] + delta[i] * w_i[-1]
    
#     if tau_perturbed[:, :, i] < B_perturbed[i] or tau_perturbed[:, :, i] < 0 or B_perturbed[i] < 0:
    tau_perturbed[:, :, i], B_perturbed[i] = proj_tau_B(T, network, tau_perturbed[:, :, i], B_perturbed[i], \
                                                    tau_max = tau_max, B_max = B_max)
    
    print("tau[:, :, i]:", tau[:, :, i])
    print("B[i]:", B[i])
    print("tau_perturbed[:, :, i]:", tau_perturbed[:, :, i])
    print("B_perturbed[i]:", B_perturbed[i])


    y_el_values, y_in_values = solve_CBCP_direct(T, network, tau = tau[:, :, i], B = B[i], v_I_array = v_I_array, \
                                                 v_E_array = v_E_array, a_ex = a, a_gp = a)

#     print()
#     print("y_el_values:", y_el_values)
#     print("y_in_values:", y_in_values)
#     print()

#     print()
#     print("New solve_CBCP_iter_11 call to solve_CBCP (perturbed):")
#     print()

    y_el_perturbed_values, y_in_perturbed_values = solve_CBCP_direct(T, network, tau = tau_perturbed[:, :, i], B = B_perturbed[i], \
                                                                     v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
    
#     print("y_el:", y_el)
#     print("y_in:", y_in)
#     print("y_el_perturbed:", y_el_perturbed)
#     print("y_in_perturbed:", y_in_perturbed)
    
    welfare = welfare_obj(T, network, lambda_E, lambda_R, lambda_I, tau = tau[:, :, i], v_I_array = v_I_array, v_E_array = v_E_array, \
                          y_el = y_el_values, y_in = y_in_values, a_ex = a, a_gp = a)
    welfare_perturbed = welfare_obj(T, network, lambda_E, lambda_R, lambda_I, tau = tau_perturbed[:, :, i], \
                                    v_I_array = v_I_array, v_E_array = v_E_array, \
                                    y_el = y_el_perturbed_values, y_in = y_in_perturbed_values, a_ex = a, a_gp = a)
    welfare_list.append(welfare)
    
#     print("welfare:", welfare)
#     print("welfare_perturbed:", welfare_perturbed)
#     print("tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:-1].reshape((network.num_edges, T)) * (welfare_perturbed - welfare):\n", \
#     tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:-1].reshape((network.num_edges, T)) * (welfare_perturbed - welfare))
    
    tau[:, :, i+1] = tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:-1].reshape((network.num_edges, T)) * (welfare_perturbed - welfare)
    
#     print()
#     print("B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare):\n", \
#          B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare))
#     print("B[i]:", B[i])
#     print("eta[i]:", eta[i])
#     print("delta[i]:", delta[i])
#     print("w_i[-1]:", w_i[-1])
#     print("welfare:", welfare)
#     print("welfare_perturbed:", welfare_perturbed)
#     print()
    
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare)
    tau[:, :, i+1], B[i+1] = proj_tau_B(T, network, tau[:, :, i+1], B[i+1], tau_max, B_max)
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.linalg.norm(tau[:, :, i-diffs_num_cols : i-1] - tau[:, :, i-diffs_num_cols+1 : i], axis = 0)
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[:, :, 0:10]:", tau[:, :, 0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:", tau_diffs)
        print("B_diffs:", B_diffs)
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            break

time_2 = time.time()

min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, :, argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print()
print("Time:", time_2 - time_1)



In [ ]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

In [ ]:
y_el_values

In [ ]:
y_in_values

## Grid Search:

In [ ]:
# tau = 0.5
# B = 0.5
# v_I = 1.0
# v_E = 0.6
# a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# y_el_point_5, y_in_point_5 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]),\
#                                                a_ex = a, a_gp = a)

# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                   y_el = y_el_point_5, y_in = y_in_point_5, \
#                                                   a_ex = a, a_gp = a)


In [ ]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I_array = np.array([1.0])
v_E_array = np.array([0.6])
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
y_init = np.array([0.0, 0.0, 1.0, 1.0, 0.0])
lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0


grid_size = 0.02
tau_arr = (np.arange(int(1/grid_size)) + 1) * grid_size
B_arr = (np.arange(int(1/grid_size)) + 1) * grid_size

welfare_obj_arr = np.ones((tau_arr.shape[0], B_arr.shape[0])) * 100
y_el_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 3))
y_in_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 2))
welfare_obj_list = []

for tau_index, tau in enumerate(tau_arr):
    for B_index, B in enumerate(B_arr):
        if B < tau:
            print("tau:", tau)
            print("B:", B)
            y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
                = solve_CBCP_direct(tau = tau, B = B,\
                                   v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
            
#             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
            welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = v_I_array, v_E_array = v_E_array, \
                                                              y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
                                                              a_ex = a, a_gp = a)
        
            welfare_obj_list.append(welfare_obj_arr[tau_index][B_index])
            
            print()

time_2 = time.time()

print("Time:", time_2 - time_1)



In [ ]:
# tau = 0.5
# B = 0.4
# v_I = 1.0
# v_E = 0.6

# y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
#                 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), a_ex = a, a_gp = a)
            
# #             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                               y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
#                                                               a_ex = a, a_gp = a)

In [ ]:
# welfare_obj_arr

argmin_indices_wrapped = np.where(welfare_obj_arr == min(welfare_obj_list))
argmin_indices = [argmin_indices_wrapped[0][0], argmin_indices_wrapped[1][0]]
# argmin_indices
argmin_tau = tau_arr[argmin_indices[0]]
argmin_B = B_arr[argmin_indices[1]]

print("argmin_tau:\n", np.round(argmin_tau, 2))
print("\nargmin_B:\n", np.round(argmin_B, 2))

welfare_obj_arr[argmin_indices[0], argmin_indices[1]]


In [ ]:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

# plt.imshow(welfare_obj_arr.T, vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 
plt.imshow(welfare_obj_arr.T, extent=[np.min(tau_arr), np.max(tau_arr), np.min(B_arr), np.max(B_arr)], \
           vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 

plt.colorbar() 
plt.xlabel("Toll")
plt.ylabel("Budget")
# plt.xticks(x_positions, x_labels)

## Test:

In [ ]:
# Test:

grad = np.array([3.11430535, 1.501, 1.501, 2.46858321, 1.501])

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_el_var = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_in_var = cp.Variable(2)

objective = cp.Minimize(grad[0:3] @ y_el_var + grad[3:] @ y_in_var)

constraints = []
constraints += [y_el_var >= 0, y_in_var >= 0]
constraints += [cp.sum(y_el_var) == 1.0, cp.sum(y_in_var) == 1.0]
constraints += [y_el_var[1] * tau <= B]

prob = cp.Problem(objective, constraints)
result = prob.solve()

print("grad:", grad)
print("y_el_var.value:", y_el_var.value)
# print("y_el_var_current:", y_el_var_current)
print("y_in_var.value:", y_in_var.value)
# print("y_in_var_current:", y_in_var_current)
print()

# y_el_var_current = y_el_var_current + 2/(k+2) * (y_el_var.value - y_el_var_current)
# y_in_var_current = y_in_var_current + 2/(k+2) * (y_in_var.value - y_in_var_current)

# y_iters[0:3, k] = y_el_var_current
# y_iters[3:, k] = y_in_var_current

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

# Solver=SCS,verbose=False

In [ ]:
# y_el_var
# y_in_var
# np.hstack((y_el_var, y_in_var))

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:

func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)
